In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


1. Separe el dataset en Train y Test (1 pt)
2. Realice Análisis de los datos (EDA) con gráficos o tablas (4 pts)
3. Grafique la matriz de correlación (2 pt)
4. Ajuste un modelo utilizando regresión logística (2 pts)
5. Muestre el accuracy del modelo utilizando tanto el train y como el test.
6. Cuales son las 2 características (features) mas relevantes a la hora de realizar la clasificación. (2 pts)
7. Cuales son las 2 características (features) menos relevantes a la hora de realizar la clasificación. (2 pts)

### 0. Carga del Dataset y revisión

In [ ]:
df=pd.read_csv("../input/paris-housing-classification/ParisHousingClass.csv")


In [ ]:
df.info()

In [ ]:
#Se revisa que no haya valores faltantes para ver si es necesario darle un tratamiento
df.isna().sum().sort_values()

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
#Se realiza una tranformación de la etiqueta mediante label Encoder

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

df['category'] = labelencoder.fit_transform(df['category'])


In [ ]:
df.head(3)

In [ ]:
# Se revisa si la clase esta balanceada

plt.figure(figsize=(8,5))
sns.countplot(df.category)
plt.title('Class distribution')
plt.show()



Se puede observar que contamos con un dataset desbalanceado, lo cual lo debemos considerar a la hora de realizar el entranamiento

### 1. División en train y test

In [ ]:
features = df.drop(["category"],axis=1)
labels = df["category"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,labels, test_size=0.30, random_state=42)

df_final=pd.DataFrame(y_test.copy())

print("Tamaño dataset entranamiento features",X_train.shape,"Tamaño dataset entranamiento labels",y_test.shape)

### 2. Análisis exploratorio de datos

Para el EDA nos vamos a centrar en el dataset de entrenamiento y si hubiese que aplicar alguna transformación o preprocesamiento se le aplica al de test también.

In [ ]:
#Se analizan variables estadísticas básicas.
X_train.describe()

Se revisará la cantidad de valores unicos por columna, ya que esto nos puede indicar que hay columnas que son atributos que podríamos tranformar mediante One Hot Encoding. Adicional de la tabla anterior se puede observar que hay varias variables binarias, como por ejemplo "hasYard","hasPool", etc

In [ ]:
for column in X_train:
    print(column,len(df[column].unique()))

Se puede observar que hasGuestRoom no es una binaria, sino que contabiliza la cantidad de cuartos para huespedes

Ahora se va a revisar si existen outliers en los features que son continuos para ver su tratamiento. En este caso se dejaran por fuera todas las features binarias

In [ ]:
#Se grafica un boxplot para ver si hay outliers muy marcados.
cont=X_train.loc[:,["squareMeters","numberOfRooms","floors","cityCode","cityPartRange","numPrevOwners","made","basement","attic","garage","hasGuestRoom","price"]]
cont2=X_test.loc[:,["squareMeters","numberOfRooms","floors","cityCode","cityPartRange","numPrevOwners","made","basement","attic","garage","hasGuestRoom","price"]]

for column in cont:
    plt.figure(column)
    sns.boxplot(data=cont,x=column)

En este caso se puede observar que no hay ningun tipo de outlier identificable en los gráficos de cajas

In [ ]:
# Se grafican las distribuciones de las variables continuas

cont.hist(bins=30, figsize=(15,15))
plt.show()

No tienen una distribución clara, para efectos de la clasificación como se utilizará una regresión logística se realizará estandarización como parte del preprocesamiento

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#Se aplica sobre dataset de test y entrenamiento
cont =scaler.fit_transform(cont.values)
cont2 =scaler.fit_transform(cont2.values)

In [ ]:
#se agregan las columnas estandarizadas al df original 
col=["squareMeters","numberOfRooms","floors","cityCode","cityPartRange","numPrevOwners","made","basement","attic","garage","hasGuestRoom","price"]
X_train[col]=cont
X_test[col]=cont2

In [ ]:
X_train.describe()

In [ ]:
X_test.describe()

### 3. Análisis de correlación 

In [ ]:
transicion=X_train.copy()
transicion["categoria"]=y_train
correlacion=transicion.corr()
plt.figure(figsize = (20,20))
sns.set(font_scale=1)
sns.heatmap(correlacion, annot = True)
plt.show()

Del grafico anterior, se puede observar que no hay una correlación lineal fuerte entre las features, solamente entre el precio y los M2, lo cual hace sentido ya que el precio normalmente se calcula por m2.

Al analizar la correlación de la etiqueta con las features, se tienen valores bajos-medios, siendo hasYard, hasPool & isNewBuilt que hace sentido al ser indicativos de que una casa es de lujo

### 4. Modelado con regresión logística y accuracy del modelo

In [ ]:
Luxury=sum(y_train==1)
total=len(y_train)

Se realiza el One Hot encoder para las variables binarias, antes de utilizar la regresion logistica

In [ ]:
X_train2=pd.DataFrame()
X_test2=pd.DataFrame()
binarias=["hasYard","hasPool","isNewBuilt","hasStormProtector","hasStorageRoom"]
X_train2 = pd.concat([X_train, pd.get_dummies(X_train,columns=binarias)], axis=1)
X_test2 = pd.concat([X_test, pd.get_dummies(X_test, columns=binarias)], axis=1)
X_train2.drop(columns=binarias, axis=1, inplace=True)
X_test2.drop(columns=binarias, axis=1, inplace=True)

In [ ]:
X_train2.head(3)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_model = LogisticRegression(class_weight="balanced")
logreg_model.fit(X_train2, y_train)


print("El accuracy del entranamiento es", logreg_model.score(X_train2, y_train))

In [ ]:
y_prediction=logreg_model.predict(X_test2)

print("El accuracy del test es", logreg_model.score(X_test2, y_test))

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

f1_test = f1_score(y_test, y_prediction)
print('The f1 score for the testing data:', f1_test)

def conf_matrix(y_test, y_prediction):    
      
    con_mat = confusion_matrix(y_test, y_prediction)
    con_mat = pd.DataFrame(con_mat, range(2), range(2))
    plt.figure(figsize=(6,6))
    sns.set(font_scale=1.5) 
    sns.heatmap(con_mat, annot=True, annot_kws={"size": 16}, fmt='g', cmap='Blues', cbar=False)
    

conf_matrix(y_test, y_prediction)

In [ ]:
df_final["prediccion"]=y_prediction
df_final

### 5. Features más y menos relevantes

Para determinar las features más y menos importantes, vamos a obtener los coeficientes. Los que de forma absoluta sean mayores, son los más importantes y los que sean menos relevantes son los que de forma absoluta tengan el valor menor.

In [ ]:
coef=pd.DataFrame(logreg_model.coef_, columns=X_train2.columns.values)
coef


In [ ]:
logreg_model.intercept_

En este caso, se puede observar que de forma general las features mas relevantes son HasYard y HasPool,que indican que si su presencia es positiva indican que es una casa lujusa. Por otro lado, las features que aportan menos al modelo y por ende son menos relevantes son numberOfRooms y garage